# 메뉴 전처리

In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from konlpy.tag import Twitter
from konlpy.tag import Kkma
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re

import warnings
warnings.filterwarnings('ignore')

In [177]:
# 한글 깨짐
import platform
if platform.system() == 'Darwin': #맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': #윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': #리눅스 (구글 콜랩)
        #!wget "https://www.wfonts.com/download/data/2016/06/13/malgun-gothic/malgun.ttf"
        #!mv malgun.ttf /usr/share/fonts/truetype/
        #import matplotlib.font_manager as fm 
        #fm._rebuild() 
        plt.rc('font', family='Malgun Gothic') 
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

In [178]:
train_raw = pd.read_csv('train.csv',encoding = 'utf-8')
test_raw = pd.read_csv('test.csv',encoding = 'utf-8')
train = train_raw
test = test_raw

# 텍스트 전처리

In [179]:
def text_preprocessing(x):
    func = '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]'
    a = re.sub("\\(.*?\\)",'',str(x)) #()안에 문자 제거
    a = re.sub(func, " ", a)
#    a = re.sub("[0-9]", "", a)
#    a = a.replace("\\n", " ")
    
    return a

In [180]:
train["중식메뉴"] = train["중식메뉴"].apply(lambda x: text_preprocessing(x))
train["석식메뉴"] = train["석식메뉴"].apply(lambda x: text_preprocessing(x))
test['중식메뉴'] = test["중식메뉴"].apply(lambda x: text_preprocessing(x))
test['석식메뉴'] = test["석식메뉴"].apply(lambda x: text_preprocessing(x))
train['중식메뉴'][0]

'쌀밥 잡곡밥  오징어찌개  쇠불고기  계란찜  청포묵무침  요구르트  포기김치  '

In [181]:
test['중식메뉴'][0:4]

0    쌀밥 흑미밥 찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...
1    쌀밥 보리밥 찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...
2    쌀밥 흑미밥 찰현미밥 팽이장국 수제돈까스 소스 가자미조림 동초나물무침 포기김치 양상...
3    쌀밥 흑미밥 찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...
Name: 중식메뉴, dtype: object

In [182]:
train_lunch = pd.DataFrame(train['중식메뉴'])
train_dinner = pd.DataFrame(train['석식메뉴'])
test_lunch = pd.DataFrame(test['중식메뉴'])
test_dinner = pd.DataFrame(test['석식메뉴'])

In [341]:
train_lunch

,중식메뉴
0,쌀밥 잡곡밥 오징어찌개 쇠불고기 계란찜 청포묵무침 요구르트 포기김치
1,쌀밥 잡곡밥 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이
2,카레덮밥 팽이장국 치킨핑거 쫄면야채무침 견과류조림 요구르트 포기김치
3,쌀밥 잡곡밥 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치
4,쌀밥 잡곡밥 떡국 돈육씨앗강정 우엉잡채 청경채무침 요구르트 포기김치
...,...
1200,쌀밥 흑미밥 찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...
1201,쌀밥 수수밥 찰현미밥 냉이된장국 동파육 봄동전 청경채 버섯숙회 초장 무생채 양상추샐...
1202,전주비빔밥 약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...
1203,쌀밥 흑미밥 찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부 달래양념장 봄...


In [190]:
#토큰화 for 불용어처리
#train_lc_tk = train_lunch['중식메뉴'].apply(lambda x: x.split())
#train_dn_tk = train_dinner['석식메뉴'].apply(lambda x: x.split())
#test_lc_tk = test_lunch['중식메뉴'].apply(lambda x: x.split())
#test_dn_tk = test_dinner['석식메뉴'].apply(lambda x: x.split())

In [188]:
#역토큰화 For 
#detokenized_doc = []
#for i in range(len(train_lunch)):
#    t = ' '.join(tokenized_doc[i])
#    detokenized_doc.append(t)
#
#train_lunch['중식메뉴'] = detokenized_doc

In [191]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = []
vectorizer = TfidfVectorizer(stop_words=stop_words, 
max_features= 2000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(train_lunch['중식메뉴'])
X.shape # TF-IDF 행렬의 크기 확인

(1205, 1609)

In [193]:
X

<1x1609 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

# 메뉴 분류
## 1.대분류 - 조리방법에 따른 분류

### 1.대분류 : 김치류
    고려한 메뉴 : 김치 깍두기 겉절이
    

In [694]:
re.findall(r"[가-힣]+김치\|[가-힣]+두기|[가-힣]+겉절이+|[가-힣]+박지",train_lunch['중식메뉴'][1])

['배추겉절이']

In [723]:
menu_dic = {'김치':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+김치|[가-힣]+두기|[가-힣]+겉절이|[가-힣]+박지",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values(): #중복메뉴 추가안할려고 했는데 작동안함.
        menu_dic['김치'].append(a)

#정규표현식에 대한 보완 필요.
#우선 현재 상황에서 위의 정규표현식이 원하는 것을 가장 많이 추출하는 표현식임.
#1. 김치로 마무리 하는 단어만 찾고 싶어서 김치$로 하면 많은 것이 빠짐. > why?
#2. 앞의 한단어가 아닌 여러단어가 있어야하는데 표현방법을 모르겠다.. ex)추출된 '신김치' 는 '신김치도토리묵'에서 추출된 단어. 

In [724]:
menu_dic

{'김치': [['포기김치'],
  ['배추겉절이'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['깍두기'],
  ['포기김치'],
  ['고등어김치', '깍두기'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['콩나물김치', '포기김치'],
  ['깍두기'],
  ['포기김치'],
  ['포기김치'],
  ['깍두기'],
  ['포기김치'],
  ['포기김치'],
  ['배추겉절이'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['등갈비김치', '열무김치'],
  ['포기김치'],
  ['포기김치'],
  ['배추겉절이'],
  ['포기김치'],
  ['돈육김치', '깍두기'],
  ['포기김치'],
  ['볶은김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['배추겉절이'],
  ['포기김치'],
  ['고등어김치', '깍두기'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['포기김치'],
  ['석박지'],
  ['포기김치'],
  ['포기김치'],
  ['깍두기'],
  ['콩나물김치', '포기김치'

In [725]:
#list형식으로 변환하여 중복되는 리스트 제거 > 진행하여도 이중리스트때문에 중복제거 한단계 더 필요
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['갓김치'],
 ['겉절이김치'],
 ['고등어김치', '깍두기'],
 ['고등어김치', '알타리김치'],
 ['고등어김치', '열무김치'],
 ['굴김치', '알타리김치'],
 ['깍두기'],
 ['꼬들빼기김치', '열무김치'],
 ['꽁치김치', '깍두기'],
 ['꽁치김치', '배추겉절이'],
 ['꽁치김치', '알타리김치'],
 ['꽁치김치', '포기김치'],
 ['꽁치캔김치', '포기김치'],
 ['꽃상추겉절이', '포기김치'],
 ['녹두김치', '포기김치'],
 ['단호박물김치'],
 ['돈갈비김치', '겉절이김치'],
 ['돈사태김치', '열무김치'],
 ['돈육김치', '깍두기'],
 ['돈육김치', '배추겉절이'],
 ['돈육김치', '백김치'],
 ['돈육김치', '부추겉절이', '깍두기'],
 ['돈육김치', '알타리김치'],
 ['돈육김치', '열무김치'],
 ['돈육김치', '포기김치'],
 ['돼지김치', '포기김치'],
 ['두부김치', '깍두기'],
 ['두부김치', '배추겉절이'],
 ['두부김치', '알타리김치'],
 ['두부김치', '열무김치'],
 ['두부김치', '포기김치'],
 ['등갈비김치', '깍두기'],
 ['등갈비김치', '배추겉절이'],
 ['등갈비김치', '알타리김치'],
 ['등갈비김치', '열무김치'],
 ['등갈비김치', '참치김치'],
 ['등갈비김치', '포기김치'],
 ['배추겉절이'],
 ['배추김치'],
 ['백김치', '포기김치'],
 ['베이컨김치', '깍두기'],
 ['베추겉절이'],
 ['보쌈김치'],
 ['볶은김치'],
 ['볶음김치'],
 ['봄동겉절이'],
 ['봄동겉절이', '깍두기'],
 ['봄동겉절이', '포기김치'],
 ['부추겉절이', '석박지'],
 ['부추겉절이', '열무김치'],
 ['부추겉절이', '포기김치'],
 ['부추팽이겉절이', '열무김치'],
 ['삼겹살김치', '포기김치'],
 ['상추겉절이'],
 ['상추겉절이', '깍두기'],
 ['상추겉절

In [726]:
#이중리스트 반환하여 중복메뉴 제거
lunch_kimchi = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_kimchi.append(j)
pd.DataFrame(set(lunch_kimchi))


,0
0,총각김치
1,해물김치
2,꽁치김치
3,신김치
4,꼬들빼기김치
5,꽁치캔김치
6,베이컨김치
7,볶음김치
8,굴김치
9,부추팽이겉절이


### 2.대분류 : 샐러드류

In [349]:
re.findall(r"[가-힣]*샐러드",train_lunch['중식메뉴'][140])

['꽃맛살오리엔탈샐러드']

In [717]:
menu_dic = {'샐러드':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]*샐러드",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['샐러드'].append(a)

In [718]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['감자고구마샐러드'],
 ['감자샐러드'],
 ['견과류샐러드'],
 ['과일샐러드'],
 ['과일요거트샐러드'],
 ['그린샐러드'],
 ['꽃맛살과일샐러드'],
 ['꽃맛살샐러드'],
 ['꽃맛살샐러드', '그린샐러드'],
 ['꽃맛살오리엔탈샐러드'],
 ['나쵸콥샐러드'],
 ['눈꽃치즈샐러드'],
 ['두릅소고기샐러드'],
 ['딸기드레싱샐러드'],
 ['로스트치킨샐러드'],
 ['마카로니콘샐러드'],
 ['만다린샐러드'],
 ['맛살떡샐러드'],
 ['멕시칸샐러드'],
 ['모듬묵샐러드'],
 ['모듬묵흑임자샐러드'],
 ['모히토과일샐러드'],
 ['버블샐러드'],
 ['베리베리샐러드'],
 ['부추샐러드', '꽃맛살샐러드'],
 ['비타민흑임자샐러드'],
 ['사과푸딩샐러드'],
 ['새싹샐러드'],
 ['샐러드'],
 ['시금치샐러드'],
 ['시리얼샐러드'],
 ['시저샐러드'],
 ['씨리얼과일샐러드'],
 ['양배추샐러드'],
 ['양상추샐러드'],
 ['양상추샐러드', '마카로니샐러드'],
 ['연어훈제샐러드'],
 ['오렌지자몽샐러드'],
 ['주꾸미세비체샐러드'],
 ['지중해샐러드'],
 ['참치야채샐러드'],
 ['치커리만다린샐러드'],
 ['치커리유자샐러드'],
 ['치킨샐러드'],
 ['치킨텐더샐러드'],
 ['카프레제샐러드'],
 ['콘샐러드'],
 ['콜리샐러드'],
 ['콥샐러드'],
 ['콩샐러드', '양상추샐러드'],
 ['크루통샐러드'],
 ['토마토샐러드'],
 ['파스타샐러드'],
 ['펜네파스타샐러드'],
 ['푸실리파스타샐러드'],
 ['풍기샐러드'],
 ['하와이안샐러드'],
 ['해초샐러드'],
 ['황도샐러드'],
 ['흑임자연근샐러드']]

In [722]:
lunch_salad = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_salad.append(j)
pd.DataFrame(set(lunch_salad))


,0
0,감자샐러드
1,멕시칸샐러드
2,씨리얼과일샐러드
3,마카로니콘샐러드
4,토마토샐러드
5,참치야채샐러드
6,두릅소고기샐러드
7,사과푸딩샐러드
8,양상추샐러드
9,꽃맛살과일샐러드


### 3.대분류 : 국수/면

In [446]:
re.findall(r"[가-힣]+국수*|[가-힣]+면+[가-힣]+",train_lunch['중식메뉴'][1061])

['냉모밀국수']

In [575]:
menu_dic = {'국수':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]*국수+[가-힣]*|[가-힣]*면+[가-힣]*",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['국수'].append(a)

In [576]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['골뱅이소면무침'],
 ['냉모밀국수'],
 ['당면채소무침'],
 ['라면땅'],
 ['라면사리'],
 ['멸치국수'],
 ['비빔메밀국수'],
 ['비빔메밀면'],
 ['소고기당면국'],
 ['소면'],
 ['소면사리'],
 ['쇠고기당면볶음'],
 ['쌀국수'],
 ['열무물국수'],
 ['잔치국수'],
 ['쫄면'],
 ['쫄면무침'],
 ['쫄면야채무침'],
 ['쫄면채소무침'],
 ['차돌비빔국수'],
 ['칼국수']]

In [577]:
lunch_noodle = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_noodle.append(j)
pd.DataFrame(set(lunch_noodle))

,0
0,칼국수
1,비빔메밀국수
2,멸치국수
3,잔치국수
4,쫄면
5,비빔메밀면
6,소면
7,열무물국수
8,골뱅이소면무침
9,소면사리


### 4.대분류 찌깨류

In [450]:
re.findall(r"[가-힣]+찌개+[가-힣]*",train_lunch['중식메뉴'][1])

['김치찌개']

In [587]:
menu_dic = {'찌개':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]*찌개+[가-힣]*",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['찌개'].append(a)

In [588]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['감자고추장찌개'],
 ['강된장찌개'],
 ['고추장찌개'],
 ['김치찌개'],
 ['꽃게된장찌개'],
 ['꽃게찌개'],
 ['냉이된장찌개'],
 ['대구찌개'],
 ['돈갈비김치찌개'],
 ['돈육김치찌개'],
 ['동태찌개'],
 ['돼지김치찌개'],
 ['된장찌개'],
 ['두부된장찌개'],
 ['민물새우찌개'],
 ['버섯들깨찌개'],
 ['부대찌개'],
 ['불낙찌개'],
 ['소고기된장찌개'],
 ['순두부찌개'],
 ['얼큰순두부찌개'],
 ['오징어찌개'],
 ['우렁된장찌개'],
 ['짜글이돼지찌개'],
 ['차돌된장찌개'],
 ['차돌박이된장찌개'],
 ['차돌박이찌개'],
 ['참치김치찌개'],
 ['청국장찌개'],
 ['청량된장찌개'],
 ['청양된장찌개'],
 ['콩비지김치찌개'],
 ['콩비지찌개'],
 ['해물된장찌개'],
 ['해물순두부찌개'],
 ['호박고추장찌개'],
 ['호박된장찌개'],
 ['호박잎된장찌개'],
 ['호박잎찌개']]

In [589]:
lunch_jjigae = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_jjigae.append(j)
pd.DataFrame(set(lunch_jjigae))

,0
0,된장찌개
1,순두부찌개
2,불낙찌개
3,호박잎된장찌개
4,해물된장찌개
5,냉이된장찌개
6,호박고추장찌개
7,동태찌개
8,김치찌개
9,짜글이돼지찌개


### 5.대분류 국탕류

In [591]:
re.findall(r"[가-힣]+국+[가-힣]*",train_lunch['중식메뉴'][2])

['팽이장국']

In [607]:
menu_dic = {'국탕':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+국+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['국탕'].append(a)

In [610]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts
#'중국' 제거 필요;;;;;;;;;;;;;
#김치와 마찬가지로 국이 마지막단어로 지정($)이 안되어서 쌀국수도 추가됌. 쌀국수 제거 필요

[[],
 ['가쯔오국'],
 ['가쯔오장국'],
 ['감자국'],
 ['감자수제비국'],
 ['감자양파국'],
 ['건새우무국'],
 ['건새우무채국'],
 ['건새우미역국'],
 ['건새우아욱국'],
 ['경상도식소고기국'],
 ['계란국'],
 ['계란파국'],
 ['굴김치두부국'],
 ['굴떡국'],
 ['굴미역국'],
 ['근대국'],
 ['근대된장국'],
 ['김치국'],
 ['김치두부국'],
 ['김치콩나물국'],
 ['김칫국'],
 ['꽃게된장국'],
 ['나가사끼짬뽕국'],
 ['냉모밀국'],
 ['냉이된장국'],
 ['냉이콩나물국'],
 ['느타리버섯국'],
 ['다슬기아욱국'],
 ['달래된장국'],
 ['도토리묵냉국'],
 ['두부김치국'],
 ['두부맑은국'],
 ['들깨무채국'],
 ['들깨미역국'],
 ['들깨버섯국'],
 ['들깨시락국'],
 ['들깨시래기국'],
 ['떡국'],
 ['떡만두국'],
 ['맑은계란국'],
 ['맑은국'],
 ['맑은순두부국'],
 ['맑은콩나물국'],
 ['매운계란파국'],
 ['매운버섯국'],
 ['매운소고기국'],
 ['매운소고기무국'],
 ['매운쇠고기국'],
 ['매운어묵국'],
 ['매운콩나물국'],
 ['매콤콩나물국'],
 ['멸치국'],
 ['무채국'],
 ['무청된장국'],
 ['물만두국'],
 ['미소국'],
 ['미소장국'],
 ['미역국'],
 ['미역미소시루국', '차돌비빔국'],
 ['미역오이냉국'],
 ['미역장국'],
 ['바지락냉이국'],
 ['바지락미역국'],
 ['바지락살무국'],
 ['배추국'],
 ['배추된장국'],
 ['버섯국'],
 ['버섯들깨국'],
 ['복숭아미역냉국'],
 ['봄동된장국'],
 ['북어계란국'],
 ['북어국'],
 ['북어채국'],
 ['북어콩나물국'],
 ['북어해장국'],
 ['북엇국'],
 ['사골떡국'],
 ['사골우거지국'],
 ['사과오이냉국'],
 ['새알만두국'],
 ['새알미역국'],
 ['새우살미역국'],
 ['소고기국'

In [611]:
lunch_soup = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_soup.append(j)
pd.DataFrame(set(lunch_soup))

,0
0,매운소고기무국
1,유부장국
2,건새우무국
3,꽃게된장국
4,조랭이떡국
...,...
174,올챙이만두국
175,미역장국
176,나가사끼짬뽕국
177,해물수제비국


### 6.장아찌류 

In [613]:
re.findall(r"[가-힣]+장아찌+",train_lunch['중식메뉴'][1038])

[]

In [616]:
menu_dic = {'장아찌':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]*장아찌+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['장아찌'].append(a)

In [617]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[], ['수제참나물장아찌'], ['장아찌']]

In [618]:
lunch_jangahjji = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_jangahjji.append(j)
pd.DataFrame(set(lunch_jangahjji))

,0
0,수제참나물장아찌
1,장아찌


### 7.구이류

In [623]:
re.findall(r"[가-힣]+구이+",train_lunch['중식메뉴'][1148])

['적어양념장구이']

In [628]:
menu_dic = {'구이':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+구이+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['구이'].append(a)
#정규표현식 이슈
#두부구이 > 새싹두부구이 인데 왜 새싹은 출력안되는지 모르겠다..

In [629]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['가래떡구이'],
 ['가자미엿장구이'],
 ['가자미엿장구이', '햇고구마오븐구이'],
 ['가지구이'],
 ['갈비구이'],
 ['갈치구이'],
 ['갈치구이', '모듬소시지구이'],
 ['감자치즈구이'],
 ['고구마오븐구이'],
 ['고구마치즈구이'],
 ['고등어구이'],
 ['고등어카레구이'],
 ['굴비구이'],
 ['굴비구이', '김구이'],
 ['김구이'],
 ['꽁치구이'],
 ['꽁치레몬구이'],
 ['꽁치오븐구이'],
 ['꽁치와사비구이'],
 ['꽁치한마리구이'],
 ['꽁치한마리레몬구이'],
 ['너비아니구이'],
 ['단호박견과류구이'],
 ['닭다리바베큐오븐구이'],
 ['닭데리야끼구이'],
 ['닭오븐구이'],
 ['더덕구이'],
 ['돈갈비양념구이'],
 ['동그랑땡구이'],
 ['두부계란구이'],
 ['두부구이'],
 ['두부새싹구이'],
 ['모둠버섯구이'],
 ['모둠소세지구이'],
 ['모둠소시지구이'],
 ['모듬버섯구이'],
 ['모듬소세지구이'],
 ['모듬소세지버섯구이'],
 ['모듬소시지구이'],
 ['목살데리야끼구이'],
 ['바베큐장각오븐구이'],
 ['버섯구이'],
 ['삼겹살구이'],
 ['삼겹살더덕고추장구이'],
 ['삼겹살오븐구이'],
 ['삼치구이'],
 ['삼치된장구이'],
 ['삼치양념구이'],
 ['새송이너비아니구이'],
 ['새송이떡갈비구이'],
 ['새싹두부구이'],
 ['소세지구이'],
 ['소세지오븐구이'],
 ['알감자버터구이'],
 ['열기어구이'],
 ['옥수수콘치즈구이'],
 ['웨지감자오븐구이'],
 ['임연수구이'],
 ['자반고등어구이'],
 ['장어고추장양념구이'],
 ['장어구이'],
 ['적어구이'],
 ['적어양념장구이'],
 ['조기구이'],
 ['콘치즈오븐구이'],
 ['통감자오븐구이'],
 ['통도라지고추장구이'],
 ['통도라지구이'],
 ['황태양념구이'],
 ['훈제오리구이'],
 ['훈제오리구이', '두부구이']]

In [630]:
lunch_grill = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_grill.append(j)
pd.DataFrame(set(lunch_grill))

,0
0,장어구이
1,가지구이
2,삼치구이
3,더덕구이
4,소세지오븐구이
...,...
63,닭다리바베큐오븐구이
64,꽁치와사비구이
65,모듬버섯구이
66,가래떡구이


### 8. 무침류

In [631]:
re.findall(r"[가-힣]+무침+",train_lunch['중식메뉴'][1148])

['통들깨부추무침']

In [634]:
menu_dic = {'무침':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+무침+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['무침'].append(a)

In [636]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['가지고추장무침'],
 ['가지무침'],
 ['건파래무침'],
 ['검은깨올방개묵무침'],
 ['고구마순무침'],
 ['고추장고구마순무침'],
 ['고추지무침'],
 ['곤약야채무침'],
 ['곤약흑임자무침'],
 ['골뱅이무침'],
 ['골뱅이무침', '세발나물무침'],
 ['골뱅이무침', '아삭고추무침'],
 ['골뱅이무침', '청포묵무침'],
 ['골뱅이무침', '취나물무침'],
 ['골뱅이무침', '콩나물무침'],
 ['골뱅이무침', '톳두부무침'],
 ['골뱅이소면무침'],
 ['골뱅이야채무침'],
 ['골뱅이채소무침'],
 ['골뱅이채소무침', '참나물무침'],
 ['깍둑오이초무침'],
 ['깻잎순무침'],
 ['꼬들단무지무침'],
 ['꼬막미나리초무침'],
 ['꽃상추무침'],
 ['꽈리고추찹쌀무침'],
 ['냉이나물무침'],
 ['냉족발야채무침'],
 ['노각무침'],
 ['단무지무침'],
 ['달래두부무침'],
 ['달래무침'],
 ['당면채소무침'],
 ['더덕무침'],
 ['도라지무침'],
 ['도라지오이무침'],
 ['도라지오이초무침'],
 ['도라지초무침'],
 ['도라지초무침', '콩나물무침'],
 ['도토리묵무침'],
 ['도토리묵무침', '물파래무침'],
 ['도토리묵야채무침'],
 ['돈나물유자청무침'],
 ['돌나물무침'],
 ['돌나물유자청무침'],
 ['두릅새송이초무침'],
 ['들깨버섯무침'],
 ['레몬미역초무침'],
 ['마늘쫑무침'],
 ['맛살겨자초무침', '노각무침'],
 ['매운어묵무침'],
 ['매운콩나물무침'],
 ['머위된장무침'],
 ['모둠채소무침'],
 ['무말랭이무침'],
 ['무채와사비무침'],
 ['문어오이미역초무침'],
 ['물미역무침'],
 ['물파래무침'],
 ['미나리오이무침'],
 ['미나리초무침', '브로콜리깨소스무침'],
 ['미나리초장무침'],
 ['미역레몬초무침'],
 ['미역무침'],
 ['미역오이초무침'],
 ['미역초무침'],
 ['버섯초무침'],
 ['봄동달래무

In [637]:
lunch_muchim = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_muchim.append(j)
pd.DataFrame(set(lunch_muchim))

,0
0,시금치초무침
1,쭈꾸미숙회무침
2,크래미숙주무침
3,돌나물유자청무침
4,숙주미나리무침
...,...
169,풋마늘초무침
170,봄동달래무침
171,도라지무침
172,도라지오이무침


### 9.볶음류

In [638]:
re.findall(r"[가-힣]+볶음+",train_lunch['중식메뉴'][1148])

['쇠고기숙주볶음']

In [655]:
menu_dic = {'볶음':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+볶음+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['볶음'].append(a)
#볶음을 마무리로 하는 달러표시$가 제대로 작동하지 않아서 닭볶음탕의 '닭볶음'이 출력됌.

In [656]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['가지볶음'],
 ['감자볶음'],
 ['감자비엔나볶음'],
 ['감자채볶음'],
 ['감자채볶음', '참치김치볶음'],
 ['감자채카레볶음'],
 ['감자채파프리카볶음'],
 ['건새우마늘쫑볶음'],
 ['건취나물볶음'],
 ['견과류멸치볶음'],
 ['계란채소볶음'],
 ['고구마순볶음'],
 ['고구마줄기볶음'],
 ['고춧잎볶음'],
 ['곤약어묵볶음'],
 ['궁중떡볶음'],
 ['김밥볶음'],
 ['김치볶음'],
 ['김치제육볶음'],
 ['꽃맛살볶음'],
 ['꽈리고추멸치볶음'],
 ['꽈리고추어묵볶음'],
 ['낙지볶음'],
 ['닭볶음'],
 ['닭볶음', '마늘쫑건새우볶음'],
 ['닭볶음', '명엽채볶음'],
 ['닭볶음', '미역줄기볶음'],
 ['닭볶음', '어묵꽈리볶음'],
 ['대패삼겹살볶음', '어묵고추장볶음'],
 ['대패삽겹숙주볶음'],
 ['도라지나물볶음'],
 ['도라지볶음'],
 ['돈육간장볶음'],
 ['돈육간장볶음', '사각어묵볶음'],
 ['돈육고추장볶음'],
 ['돈육굴소스볶음'],
 ['돈육굴소스볶음', '감자채볶음'],
 ['돈육굴소스볶음', '마늘쫑건새우볶음'],
 ['돈육굴소스볶음', '매운어묵볶음'],
 ['돈육김치볶음'],
 ['돈육김치볶음', '궁중떡볶음'],
 ['돈육김치볶음', '사각어묵볶음'],
 ['돈육꽈리고추볶음', '매운어묵볶음'],
 ['돈육도라지고추장볶음'],
 ['돈육볶음'],
 ['돈육피망볶음'],
 ['돼지고기유자청볶음'],
 ['두반장가지볶음'],
 ['두반장감자볶음'],
 ['땅콩멸치볶음'],
 ['마늘쫑건새우볶음'],
 ['마늘쫑볶음'],
 ['매운소고기낙지볶음'],
 ['매운쇠고기버섯볶음'],
 ['매운어묵볶음'],
 ['매운족발볶음'],
 ['매운주꾸미볶음'],
 ['매운호박볶음'],
 ['매콤낙지볶음'],
 ['매콤볼어묵볶음'],
 ['매콤해물볶음'],
 ['머위대들깨볶음'],
 ['멸치볶음'],
 ['멸치아몬드볶음'],
 ['멸치크랜베리볶음'],
 ['멸치크렌

In [657]:
lunch_bokkem = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_bokkem.append(j)
pd.DataFrame(set(lunch_bokkem))

,0
0,햄감자채볶음
1,모듬소시지볶음
2,소세지피망볶음
3,비엔케찹볶음
4,오이볶음
...,...
138,닭볶음
139,불닭볶음
140,두반장감자볶음
141,궁중떡볶음


### 10.전류

In [659]:
re.findall(r"[가-힣]+전+",train_lunch['중식메뉴'][455])

['부추전']

In [660]:
menu_dic = {'전':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+전+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['전'].append(a)

In [661]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['감자전'],
 ['감자채전'],
 ['건새우호박채전'],
 ['고기완자전'],
 ['고기전'],
 ['김치고기전'],
 ['김치전'],
 ['깻잎전'],
 ['꼬지삼색전'],
 ['녹두김치전'],
 ['녹두전'],
 ['늙은호박전'],
 ['단호박채소전'],
 ['동그랑땡전'],
 ['동태전'],
 ['동태전', '새송이전'],
 ['동태포전'],
 ['두부계란전'],
 ['두부맛전'],
 ['맛살전'],
 ['메밀버섯전'],
 ['메밀부추전'],
 ['메밀전'],
 ['물파래전'],
 ['버섯메밀전'],
 ['봄동전'],
 ['부추고추전'],
 ['부추깻잎전'],
 ['부추전'],
 ['부추호박전'],
 ['분홍소시지전', '새송이전'],
 ['사우전'],
 ['삼색꼬지전'],
 ['새송이버섯전'],
 ['새송이전'],
 ['스팸계란전'],
 ['애호박전'],
 ['옥수수전'],
 ['완자전'],
 ['육전'],
 ['참치두부전'],
 ['참치야채전'],
 ['청양부추전'],
 ['파프리카계란전'],
 ['해물김치전'],
 ['해물부추전'],
 ['해물완자전'],
 ['해물전'],
 ['해물파전'],
 ['호박부추전'],
 ['호박전']]

In [662]:
lunch_jeon = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_jeon.append(j)
pd.DataFrame(set(lunch_jeon))

,0
0,해물완자전
1,완자전
2,참치야채전
3,맛살전
4,두부계란전
5,파프리카계란전
6,해물전
7,메밀버섯전
8,늙은호박전
9,두부맛전


### 11.조림류

In [664]:
re.findall(r"[가-힣]+조림+",train_lunch['중식메뉴'][450])

['메추리알조림']

In [667]:
menu_dic = {'조림':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+조림+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['조림'].append(a)

In [668]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['가자미무조림'],
 ['가자미엿장조림'],
 ['간장두부조림'],
 ['갈치감자조림'],
 ['갈치양념조림'],
 ['갈치조림'],
 ['감자조림'],
 ['검정콩조림'],
 ['견과류연근조림'],
 ['견과류조림'],
 ['계란버섯장조림'],
 ['계란장조림'],
 ['고등어조림'],
 ['고추장감자조림'],
 ['곤약메추리알조림'],
 ['꽁치캔김치조림'],
 ['꽈리고추감자조림'],
 ['꽈리고추메추리알조림'],
 ['꽈리고추어묵조림'],
 ['단호박장조림'],
 ['닭가슴살장조림'],
 ['닭간장조림'],
 ['닭감자조림'],
 ['닭데리야끼조림'],
 ['닭양념조림'],
 ['돈육장조림'],
 ['동태무조림'],
 ['두부양념조림'],
 ['두부조림'],
 ['두부참치조림'],
 ['땅콩조림'],
 ['떡갈비조림'],
 ['마파무조림'],
 ['매콤떡갈비조림'],
 ['메추리알곤약조림'],
 ['메추리알조림'],
 ['명태양념조림'],
 ['명태엿장조림'],
 ['명태조림'],
 ['명태코다리조림'],
 ['모둠장조림'],
 ['모듬장조림'],
 ['문어꽈리고추조림'],
 ['미니함박조림'],
 ['미트볼조림'],
 ['미트볼칠리조림'],
 ['미트볼케찹조림'],
 ['버섯메추리알장조림'],
 ['브로콜리새송이메추리알조림'],
 ['비엔나감자조림'],
 ['비엔나떡조림'],
 ['비엔나컬리플라워조림'],
 ['비엔나케찹조림'],
 ['삼치무조림'],
 ['새송이버섯조림'],
 ['소고기장조림'],
 ['소고기장조림', '갈치무조림'],
 ['소고기장조림', '명태조림'],
 ['소세지감자조림'],
 ['쇠고기단호박조림'],
 ['쇠고기모듬장조림'],
 ['쇠고기장조림'],
 ['시래기조림'],
 ['양념두부조림'],
 ['어묵간장조림'],
 ['어묵꽈리고추조림'],
 ['연근땅콩조림'],
 ['연근조림'],
 ['우엉조림'],
 ['임연수무조림'],
 ['찰떡떡갈비조림'],
 ['코다리양념조림'],
 ['코다리조림'],
 ['콩조림']]

In [669]:
lunch_jolim = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_jolim.append(j)
pd.DataFrame(set(lunch_jolim))

,0
0,가자미엿장조림
1,명태양념조림
2,꽈리고추감자조림
3,명태엿장조림
4,미트볼케찹조림
...,...
68,코다리양념조림
69,비엔나케찹조림
70,미트볼조림
71,메추리알조림


### 12. 찜류

In [673]:
re.findall(r"[가-힣]+찜+",train_lunch['중식메뉴'][453])

['계란찜']

In [674]:
menu_dic = {'찜':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+찜+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['찜'].append(a)

In [675]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['가래떡돼지갈비찜'],
 ['가자미양념찜'],
 ['가지찜'],
 ['간장돼지갈비찜'],
 ['간장찜'],
 ['계란찜'],
 ['고등어김치말이찜'],
 ['궁중떡찜'],
 ['깻잎찜'],
 ['꽁치김치말이찜'],
 ['꽈리고추찜'],
 ['날치알계란찜'],
 ['단호박계란찜'],
 ['닭매운찜'],
 ['닭찜'],
 ['돈갈비찜'],
 ['돈사태김치찜'],
 ['돼지갈비찜'],
 ['두부고기양념찜'],
 ['등갈비김치찜'],
 ['만두찜'],
 ['맛살계란찜'],
 ['매운닭찜'],
 ['매운돈갈비찜'],
 ['매운동태찜'],
 ['매운돼지갈비찜'],
 ['매콤돼지갈비찜'],
 ['묵은지닭찜'],
 ['봉추찜'],
 ['브로컬리들깨찜'],
 ['삼겹살김치찜'],
 ['소갈비찜'],
 ['쇠고기두부찜'],
 ['아귀순살찜'],
 ['아귀콩나물찜'],
 ['안동찜'],
 ['안동찜', '꽈리고추찜'],
 ['야채계란찜'],
 ['양념돼지갈비찜'],
 ['양념찜'],
 ['연두부찜'],
 ['옥수수계란찜'],
 ['임연수찜'],
 ['청경채찜'],
 ['치즈계란찜'],
 ['카레닭찜'],
 ['콩나물동태찜'],
 ['한방소갈비찜'],
 ['해물돼지갈비찜'],
 ['해물섞어찜'],
 ['해물콩나물찜']]

In [676]:
lunch_jjim = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_jjim.append(j)
pd.DataFrame(set(lunch_jjim))

,0
0,단호박계란찜
1,닭찜
2,만두찜
3,고등어김치말이찜
4,소갈비찜
5,매운돈갈비찜
6,계란찜
7,브로컬리들깨찜
8,매콤돼지갈비찜
9,가자미양념찜


### 13. 튀김류

In [679]:
re.findall(r"[가-힣]+튀김+",train_lunch['중식메뉴'][546])

['가자미카레튀김']

In [728]:
menu_dic = {'튀김':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+튀김+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['튀김'].append(a)

In [729]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['가자미카레튀김'],
 ['가자미튀김'],
 ['가지완자튀김'],
 ['고구마튀김'],
 ['고추튀김'],
 ['김말이튀김'],
 ['모듬튀김'],
 ['새우튀김'],
 ['야채튀김'],
 ['양파링카레튀김'],
 ['오징어튀김'],
 ['춘권튀김'],
 ['캔꽁치튀김']]

In [730]:
lunch_fried = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_fried.append(j)
pd.DataFrame(set(lunch_fried))

,0
0,캔꽁치튀김
1,양파링카레튀김
2,오징어튀김
3,새우튀김
4,춘권튀김
5,김말이튀김
6,고구마튀김
7,고추튀김
8,모듬튀김
9,야채튀김


### 14.유제품(요구르트만 있음, 우유 없음)

In [734]:
re.findall(r"[가-힣]*요구르트+",train_lunch['중식메뉴'][1062])

['요구르트']

In [735]:
menu_dic = {'요구르트':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]*요구르트+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['요구르트'].append(a)

In [736]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[], ['요구르트'], ['요구르트', '요구르트']]

In [738]:
lunch_yogurt = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_yogurt.append(j)
pd.DataFrame(set(lunch_yogurt))

,0
0,요구르트


### 15. 빵/과자류 (빵만 있음)

In [741]:
re.findall(r"[가-힣]*빵+",train_lunch['중식메뉴'][136])

['꽃빵']

In [742]:
menu_dic = {'빵':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]*빵+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['빵'].append(a)

In [743]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[], ['고구마치즈빵'], ['꽃빵'], ['붕어빵']]

In [744]:
lunch_bread = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_bread.append(j)
pd.DataFrame(set(lunch_bread))

,0
0,고구마치즈빵
1,붕어빵
2,꽃빵


### 16. 음료 및 주류 - 음료,주스, 식혜,

In [746]:
re.findall(r"[가-힣]*음료+|[가-힣]*주스+|[가-힣]*식혜+",train_lunch['중식메뉴'][136])

[]

In [747]:
menu_dic = {'음료':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]*음료+|[가-힣]*주스+|[가-힣]*식혜+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['음료'].append(a)

In [748]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['매실음료'],
 ['매실주스'],
 ['배도라지주스'],
 ['사과나무주스'],
 ['살구복숭아주스'],
 ['수제단호박식혜'],
 ['식혜'],
 ['알로에주스'],
 ['오렌지주스'],
 ['오미자주스'],
 ['유기농식혜'],
 ['음료'],
 ['청포도주스'],
 ['포도주스']]

In [749]:
lunch_drink = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_drink.append(j)
pd.DataFrame(set(lunch_drink))

,0
0,청포도주스
1,오렌지주스
2,사과나무주스
3,살구복숭아주스
4,알로에주스
5,오미자주스
6,음료
7,수제단호박식혜
8,매실주스
9,식혜


### 17. 김밥류 - total 1day

In [751]:
re.findall(r"김밥",train_lunch['중식메뉴'][136])

[]

In [752]:
menu_dic = {'김밥':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"김밥",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['김밥'].append(a)

In [754]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[], ['김밥']]

### 18. 덮밥류

In [757]:
re.findall(r"[가-힣]+덮밥+",train_lunch['중식메뉴'][1008])

['마파두부덮밥']

In [758]:
menu_dic = {'덮밥':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+덮밥+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['덮밥'].append(a)

In [759]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['김치제육덮밥'],
 ['돈육버섯고추장덮밥'],
 ['마파두부덮밥'],
 ['불고기덮밥'],
 ['소불고기덮밥'],
 ['연복풍덮밥'],
 ['짜장덮밥'],
 ['참치회덮밥'],
 ['카레덮밥']]

In [761]:
lunch_ricebowl = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_ricebowl.append(j)
pd.DataFrame(set(lunch_ricebowl))

,0
0,소불고기덮밥
1,마파두부덮밥
2,카레덮밥
3,돈육버섯고추장덮밥
4,불고기덮밥
5,김치제육덮밥
6,짜장덮밥
7,참치회덮밥
8,연복풍덮밥


### 19. 비빔밥류

In [763]:
re.findall(r"[가-힣]+비빔밥+",train_lunch['중식메뉴'][1023])

['낙지비빔밥']

In [767]:
menu_dic = {'비빔밥':[]}
for i in range(0,len(train_lunch)):
    a = re.findall(r"[가-힣]+비빔밥+",train_lunch['중식메뉴'][i])
    if a not in menu_dic.values():
        menu_dic['비빔밥'].append(a)

In [768]:
list_of_unique_dicts=list(np.unique(np.array(list(menu_dic.values()))))
list_of_unique_dicts

[[],
 ['건강비빔밥'],
 ['나물비빔밥'],
 ['낙지비빔밥'],
 ['바지락비빔밥'],
 ['봄나물비빔밥'],
 ['봄새싹비빔밥'],
 ['불고기비빔밥'],
 ['산채비빔밥'],
 ['열무보리비빔밥'],
 ['열무비빔밥'],
 ['전주비빔밥']]

In [770]:
lunch_bibimbap = [] 
for i in list_of_unique_dicts:        # dic에서 안쪽 리스트를 꺼냄
    for j in i:    # 안쪽 리스트에서 요소를 하나씩 꺼냄
        lunch_bibimbap.append(j)
pd.DataFrame(set(lunch_bibimbap))

,0
0,열무비빔밥
1,건강비빔밥
2,봄새싹비빔밥
3,산채비빔밥
4,불고기비빔밥
5,열무보리비빔밥
6,낙지비빔밥
7,바지락비빔밥
8,나물비빔밥
9,전주비빔밥


### 20. 죽류 total 1day index1184 - 새알팥죽